In [8]:
from dotenv import load_dotenv
load_dotenv()
from langchain_cohere import ChatCohere
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [2]:
model = ChatCohere(model="command-r")

# Making the model _stateful_

In [4]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

In [5]:
store = {} # Will store the messages against sessions id

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [7]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [10]:
config = {
    "configurable": {
        "session_id": "a2b3c",
    },
}

response = with_message_history.invoke(
    [HumanMessage("Hi! I'm Mark.")],
    config=config
)

response.content

"Hi Mark! It's nice to meet you. How's it going today?"

In [11]:
response = with_message_history.invoke(
    [HumanMessage("What's my name?")],
    config=config
)

response.content

'Your name is Mark! As an AI chatbot, I deduced that from the first message you sent me.'

In [12]:
store

{'a2b3c': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Mark."), AIMessage(content="Hi Mark! It's nice to meet you. How's it going today?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '37eef3a3-c569-4606-9c40-d36ea7a195ad', 'token_count': {'input_tokens': 72, 'output_tokens': 16}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '37eef3a3-c569-4606-9c40-d36ea7a195ad', 'token_count': {'input_tokens': 72, 'output_tokens': 16}}, id='run-8110dc72-f8c3-4d67-8bdf-3e79428b8e9d-0'), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Mark! As an AI chatbot, I deduced that from the first message you sent me.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generat

In [15]:
response = with_message_history.invoke(
    [HumanMessage("What's my name?")],
    config={"configurable":{"session_id": "e2f3g"}} # Passing a different config
)
response.content

'Your name is Sofia. Is there anything else I can help you with?'

In [16]:
store

{'a2b3c': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Mark."), AIMessage(content="Hi Mark! It's nice to meet you. How's it going today?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '37eef3a3-c569-4606-9c40-d36ea7a195ad', 'token_count': {'input_tokens': 72, 'output_tokens': 16}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '37eef3a3-c569-4606-9c40-d36ea7a195ad', 'token_count': {'input_tokens': 72, 'output_tokens': 16}}, id='run-8110dc72-f8c3-4d67-8bdf-3e79428b8e9d-0'), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Mark! As an AI chatbot, I deduced that from the first message you sent me.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generat

# Persisting state across runs

In [30]:
import json
from langchain.schema import messages_to_dict, messages_from_dict

test_obj = {"test": "dictionary"}
session_id = "abcd"
# json.dump(test_obj, open(f"{session_id}.txt", "r"))

In [39]:
message_history = [
    HumanMessage("Hello! I am Mark."),
    AIMessage("Hi Mark!"),
]
message_history = InMemoryChatMessageHistory(messages=message_history)

temp_dict = messages_to_dict(message_history.messages)

[{'type': 'human', 'data': {'content': 'Hello! I am Mark.', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': None, 'example': False}}, {'type': 'ai', 'data': {'content': 'Hi Mark!', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': None, 'example': False, 'tool_calls': [], 'invalid_tool_calls': [], 'usage_metadata': None}}]


langchain_core.messages.human.HumanMessage

In [57]:
import json

current_sessions = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in current_sessions:
        try:
            with open(f"{session_id}.txt", "r") as f:
                current_sessions.update(
                    messages_from_dict(json.loads(f))
                )
        except:
            current_sessions[session_id] = InMemoryChatMessageHistory()
            with open(f"{session_id}.txt", "w") as f:
                json.dump({}, f)
    else:
        with open(f"{session_id}.txt", "w") as f:
            json.dump(
                messages_to_dict(current_sessions[session_id].messages),
                f,
            )
    return current_sessions[session_id]

In [58]:
config = {
    "configurable": {
        "session_id": "a2b3c",
    },
}

with_persistent_history = RunnableWithMessageHistory(model, get_session_history)

In [59]:
def talk(text: str):
    response = with_persistent_history.invoke(
        [HumanMessage(text)],
        config=config
    )
    return response.content

In [60]:
talk("Hello, my name's Mark")

"Hi Mark! It's nice to meet you. How's it going today?"

In [61]:
talk("Good! Thanks for asking. Wbu?")

"I'm doing well, thank you for asking! I'm having a great day so far and I'm happy to be of assistance to you. As an AI chatbot, I don't have any days, but I'm always ready to help and chat!"

In [64]:
talk("What's the opposite of top?")

"The opposite of 'top' could be 'bottom'. The words are often used to describe relative positions, with one being at the highest point and the other at the lowest point. \n\nAnother opposite could be the word 'worst', as the top is usually associated with the best or highest quality, while the worst indicates the lowest or poor quality."

In [65]:
with open("a2b3c.txt", "r") as f:
    messages = messages_from_dict(json.load(f))
for m in messages:
    print(m.content)

Hello, my name's Mark
Hi Mark! It's nice to meet you. How's it going today?
Good! Thanks for asking. Wbu?
I'm doing well, thank you for asking! I'm having a great day so far and I'm happy to be of assistance to you. As an AI chatbot, I don't have any days, but I'm always ready to help and chat!
